# Trabajo Final - Tostadora "Campesino"

## Autor: Leonard David Vivas Dallos

## Objetivo
Predecir la calidad del café basados en el Puntaje de Taza.

## Indicaciones

1. Analizar los datos y definir las variables que se utilizarán de input. Tenga cuidad que no se filtre información al momento de elegirlas.
2. Definir el preprocesamiento que se debe hacer tanto para las X como para la Y.
3. Entrenar al menos dos modelos de regresión.
4. Utilizar un método de explicabilidad (LIME, SHAP, Feature Importance)
5. Reportar resultados.

El paso inicial es cargar los datos y realizar un análisis exploratorio para entender las variables disponibles y su relación con el Puntaje de Taza. Como sabemos, la información brindada para este análisis viene en un archivo de Excel, .xlsx, que contiene la información necesaria para el análisis. En primer lugar, cargaremos los datos, y los revisaremos para identificar las variables relevantes y como podemos relacionarlas con el Puntaje de Taza.

In [33]:
# Cargar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

### Archivo "CC FT 17   Formato de Control de Calidad Café de Trillado (1).xlsx"

In [34]:
# Inicialmente, cargamos los datos desde el archivo Excel
data_control_calidad = pd.read_excel("CC FT 17   Formato de Control de Calidad Café de Trillado (1).xlsx", skiprows=7)
# Revisamos las primeras filas del DataFrame para entender su estructura
data_control_calidad.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,%,C/NC,#,C/NC.1,C/NC.2,Unnamed: 9,N°,C/NC.3,SI/NO,Unnamed: 13
0,19-07-22,01-190722,Madre Laura,765.00,10.9,C,14.0,C,C,"Chocolate negro, toque frutal, cuerpo medio, a...",84,C,Si,LFQ
1,19-7-22,09-190722,Tabi Natural,204.00,10.2,C,14.0,C,C,"Frutas maduras, nibs de cacao, acidez brillant...",85,C,Si,LFQ
2,19-07-22,10-190722,Don Mario,165.00,10.7,C,14.0,C,C,"Panela, durazno, miel, acidez brillante citric...",84.5,C,Si,LFQ
3,27-07-22,07-19-07-22,Don Felix,0.45,10.5,C,14.0,C,C,"Moras maduras, chocolate negro, acidez media c...",84.5,C,Si,LFQ
4,31-10-22,01-291022,Madre Laura,105.00,10.7,C,14.0,C,C,"Chocolate negro, toque frutal, cuerpo medio, a...",84,C,Si,LFQ


Dado un vistazo inicial a los datos, por la configuración del archivo de Excel, tenemos que saltar las primeras 7 filas para obtener el DataFrame correctamente, sin embargo, aún tendremos que asignar manualmente los nombres de las columnas, ya que el archivo no los contiene en la fila 8 (por problemas de formato de filas combinadas).

In [ ]:
# Asignamos manualmente los nombres de las columnas
data_control_calidad.columns = [
    'Fecha', 'Lote', 'Denominación', 'Cantidad', '% Humedad', 'C/NC Humedad', 'Mallas', 'C/NC Mallas', 'Verificación Física Café',
    'Notas de Catación', 'Puntaje de Taza', 'C/NC Puntaje de Taza', 'Liberación de Lote(Si/No)', 'Responsable'
]
# Revisamos nuevamente las primeras filas del DataFrame con los nombres de columnas actualizados
data_control_calidad.head()

,Fecha,Lote,Denominación,Cantidad,% Humedad,C/NC Humedad,Mallas,C/NC Mallas,Verificación Física Café,Notas de Catación,Puntaje de Taza,C/NC Puntaje de Taza,Liberación de Lote(Si/No),Responsable
0,19-07-22,01-190722,Madre Laura,765.00,10.9,C,14.0,C,C,"Chocolate negro, toque frutal, cuerpo medio, a...",84,C,Si,LFQ
1,19-7-22,09-190722,Tabi Natural,204.00,10.2,C,14.0,C,C,"Frutas maduras, nibs de cacao, acidez brillant...",85,C,Si,LFQ
2,19-07-22,10-190722,Don Mario,165.00,10.7,C,14.0,C,C,"Panela, durazno, miel, acidez brillante citric...",84.5,C,Si,LFQ
3,27-07-22,07-19-07-22,Don Felix,0.45,10.5,C,14.0,C,C,"Moras maduras, chocolate negro, acidez media c...",84.5,C,Si,LFQ
4,31-10-22,01-291022,Madre Laura,105.00,10.7,C,14.0,C,C,"Chocolate negro, toque frutal, cuerpo medio, a...",84,C,Si,LFQ


In [47]:
data_control_calidad['Fecha'] = pd.to_datetime(data_control_calidad['Fecha'], dayfirst=True, errors='coerce')

data_control_calidad.head()

,Fecha,Lote,Denominación,Cantidad,% Humedad,C/NC Humedad,Mallas,C/NC Mallas,Verificación Física Café,Notas de Catación,Puntaje de Taza,C/NC Puntaje de Taza,Liberación de Lote(Si/No),Responsable
0,2022-07-19,01-190722,Madre Laura,765.00,10.9,C,14.0,C,C,"Chocolate negro, toque frutal, cuerpo medio, a...",84,C,Si,LFQ
1,2022-07-19,09-190722,Tabi Natural,204.00,10.2,C,14.0,C,C,"Frutas maduras, nibs de cacao, acidez brillant...",85,C,Si,LFQ
2,2022-07-19,10-190722,Don Mario,165.00,10.7,C,14.0,C,C,"Panela, durazno, miel, acidez brillante citric...",84.5,C,Si,LFQ
3,2022-07-27,07-19-07-22,Don Felix,0.45,10.5,C,14.0,C,C,"Moras maduras, chocolate negro, acidez media c...",84.5,C,Si,LFQ
4,2022-10-31,01-291022,Madre Laura,105.00,10.7,C,14.0,C,C,"Chocolate negro, toque frutal, cuerpo medio, a...",84,C,Si,LFQ


#### Archivo "CC FT 18  Formato de  Tostión (1).xlsx"

In [36]:
data_tostion = pd.read_excel("CC FT 18  Formato de  Tostión (1).xlsx", skiprows=5)

data_tostion.head()

,Fecha,Lote,Origen,Variedad,Proceso,Beneficio,Peso en Verde,Merma,Peso en Tostado,Perfil,Temp. De inicio y final,Tiempo de tueste,Observaciones,Tostador
0,22-07-22,01-190722,Jerico,Dos mil,Tradicional,Lavado,9.00,15.000000,7.65,Filtrado,175°/191°,08:01:00,NaN,LFQ
1,25-07-22,01-190722,Jerico,Dos mil,Tradicional,Lavado,3.00,16.666667,2.50,Espressso,175°/195°,08:42:00,NaN,LFQ
2,25-07-22,01-190722,Jerico,Dos mil,Tradicional,Lavado,9.00,16.444444,7.52,Filtrado,175°/190°,07:58:00,NaN,LFQ
3,28-07-22,01-190722,Jerico,Dos mil,Tradicional,Lavado,11.70,16.324786,9.79,Filtrado,175°/191°,08:02:00,NaN,LFQ
4,28-07-22,09-190722,Ciudad Bolivar,Tabi,Natural,Natural,0.45,24.444444,0.34,Filtrado,150°/186°,08:10:00,NaN,LFQ


Podemos ver que la columna de "Temp. De inicio y final" contiene dos valores separados por un /, lo que indica que debemos dividir esta columna en dos columnas separadas: "Temp. De inicio" y "Temp. Final". Esto nos permitirá trabajar con los datos de temperatura de manera más efectiva.

In [37]:
# Dividimos la columna "Temp. De inicio y final" en dos columnas separadas
data_tostion[['Temp. De inicio', 'Temp. Final']] = data_tostion['Temp. De inicio y final'].str.split('/', expand=True)
# Eliminamos la columna original "Temp. De inicio y final"
data_tostion.drop(columns=['Temp. De inicio y final'], inplace=True)
# Revisamos las primeras filas del DataFrame de tostión
data_tostion.head()

,Fecha,Lote,Origen,Variedad,Proceso,Beneficio,Peso en Verde,Merma,Peso en Tostado,Perfil,Tiempo de tueste,Observaciones,Tostador,Temp. De inicio,Temp. Final
0,22-07-22,01-190722,Jerico,Dos mil,Tradicional,Lavado,9.00,15.000000,7.65,Filtrado,08:01:00,NaN,LFQ,175°,191°
1,25-07-22,01-190722,Jerico,Dos mil,Tradicional,Lavado,3.00,16.666667,2.50,Espressso,08:42:00,NaN,LFQ,175°,195°
2,25-07-22,01-190722,Jerico,Dos mil,Tradicional,Lavado,9.00,16.444444,7.52,Filtrado,07:58:00,NaN,LFQ,175°,190°
3,28-07-22,01-190722,Jerico,Dos mil,Tradicional,Lavado,11.70,16.324786,9.79,Filtrado,08:02:00,NaN,LFQ,175°,191°
4,28-07-22,09-190722,Ciudad Bolivar,Tabi,Natural,Natural,0.45,24.444444,0.34,Filtrado,08:10:00,NaN,LFQ,150°,186°


Adicionalmente, revisemos los nombres de las columnas del archivo de tostión para asegurarnos de que están correctamente asignados y no contienen espacios adicionales o caracteres no deseados.

In [44]:
print(data_tostion.columns.tolist())

['Fecha ', 'Lote ', 'Origen', 'Variedad', 'Proceso', 'Beneficio ', 'Peso en Verde', 'Merma ', 'Peso en Tostado ', 'Perfil ', 'Tiempo de tueste', 'Observaciones ', 'Tostador', 'Temp. De inicio', 'Temp. Final']


Como presentiamos, las columnas tienen espacios adicionales, lo que puede causar problemas al acceder a ellas. Por lo tanto, es importante eliminar estos espacios para evitar errores en el análisis posterior. Aparte de hacer esto, formatearemos las fechas para que todas tengan el mismo formato, lo que facilitará su manejo y análisis posterior.

In [45]:
data_tostion.columns = data_tostion.columns.str.strip()

data_tostion['Fecha'] = pd.to_datetime(data_tostion['Fecha'], dayfirst=True, errors='coerce')

data_tostion.head()

,Fecha,Lote,Origen,Variedad,Proceso,Beneficio,Peso en Verde,Merma,Peso en Tostado,Perfil,Tiempo de tueste,Observaciones,Tostador,Temp. De inicio,Temp. Final
0,2022-07-22,01-190722,Jerico,Dos mil,Tradicional,Lavado,9.00,15.000000,7.65,Filtrado,08:01:00,NaN,LFQ,175°,191°
1,2022-07-25,01-190722,Jerico,Dos mil,Tradicional,Lavado,3.00,16.666667,2.50,Espressso,08:42:00,NaN,LFQ,175°,195°
2,2022-07-25,01-190722,Jerico,Dos mil,Tradicional,Lavado,9.00,16.444444,7.52,Filtrado,07:58:00,NaN,LFQ,175°,190°
3,2022-07-28,01-190722,Jerico,Dos mil,Tradicional,Lavado,11.70,16.324786,9.79,Filtrado,08:02:00,NaN,LFQ,175°,191°
4,2022-07-28,09-190722,Ciudad Bolivar,Tabi,Natural,Natural,0.45,24.444444,0.34,Filtrado,08:10:00,NaN,LFQ,150°,186°


Observemos que la columna "Observaciones" al parecer solo contiene NaN, lo que indica que no hay información en esta. Confirmemos esto y, si es así, eliminaremos esta columna del DataFrame para evitar confusiones en el análisis posterior.

In [48]:
# Revisar valores de "Observaciones" para confirmar si contiene solo NaN
observaciones_nulos = data_tostion['Observaciones'].isnull().all()
if observaciones_nulos:
    # Si la columna "Observaciones" contiene solo NaN, la eliminamos
    data_tostion.drop(columns=['Observaciones'], inplace=True)
else:
    print("La columna 'Observaciones' contiene datos y no se eliminará.")

In [49]:
data_tostion.head()

,Fecha,Lote,Origen,Variedad,Proceso,Beneficio,Peso en Verde,Merma,Peso en Tostado,Perfil,Tiempo de tueste,Tostador,Temp. De inicio,Temp. Final
0,2022-07-22,01-190722,Jerico,Dos mil,Tradicional,Lavado,9.00,15.000000,7.65,Filtrado,08:01:00,LFQ,175°,191°
1,2022-07-25,01-190722,Jerico,Dos mil,Tradicional,Lavado,3.00,16.666667,2.50,Espressso,08:42:00,LFQ,175°,195°
2,2022-07-25,01-190722,Jerico,Dos mil,Tradicional,Lavado,9.00,16.444444,7.52,Filtrado,07:58:00,LFQ,175°,190°
3,2022-07-28,01-190722,Jerico,Dos mil,Tradicional,Lavado,11.70,16.324786,9.79,Filtrado,08:02:00,LFQ,175°,191°
4,2022-07-28,09-190722,Ciudad Bolivar,Tabi,Natural,Natural,0.45,24.444444,0.34,Filtrado,08:10:00,LFQ,150°,186°


#### Archivo "CC FT 21   Formato de Control de Despachos (1).xlsx"

In [38]:
data_despachos = pd.read_excel("CC FT 21   Formato de Control de Despachos (1).xlsx", skiprows=5)

# Eliminamos la primera fila que contiene el encabezado duplicado
data_despachos = data_despachos.iloc[1:]

data_despachos.head()

,# PEDIDO,FECHA DE TUESTE,FECHA DE EMPAQUE,TIPO DE CAFÉ,PRESENTACIÓN,Unnamed: 5,CANTIDAD,Unnamed: 7,Unnamed: 8,CLIENTE,RESPONSABLE DESPACHO,VERIFICA
1,564,22-7-22,25-7-22,Madre Laura,500 Gr,NaN,3.0,NaN,NaN,Natalia Londoño,LFQ,MS
2,564,22-7-22,25-7-22,Madre Laura,1000 Gr,NaN,1.0,NaN,NaN,Natalia Londoño,LFQ,MS
3,566,22-7-22,25-7-22,Madre Laura,2500 Gr,NaN,2.0,NaN,NaN,Mateo Duque,LFQ,MS
4,567,22-7-22,26-7-22,Madre Laura,2500 Gr,NaN,1.0,NaN,NaN,Akio,LFQ,MS
5,568,25-7-22,26-7-22,Madre Laura,2500 Gr,NaN,3.0,NaN,NaN,Juan Camilo Baez,LFQ,MS


En un primer vistazo, podemos ver que tenemos ciertas columnas que no traen información alguna, esto debido al manejo de las columnas combinadas en el archivo de Excel. Por lo tanto, debemos eliminar las columnas que no contienen información relevante para nuestro análisis.

In [39]:
# Eliminamos las columnas con nombre Unnamed que no contienen información relevante
data_despachos = data_despachos.loc[:, ~data_despachos.columns.str.contains('^Unnamed')]
# Revisamos las primeras filas del DataFrame de despachos
data_despachos.head()

,# PEDIDO,FECHA DE TUESTE,FECHA DE EMPAQUE,TIPO DE CAFÉ,PRESENTACIÓN,CANTIDAD,CLIENTE,RESPONSABLE DESPACHO,VERIFICA
1,564,22-7-22,25-7-22,Madre Laura,500 Gr,3.0,Natalia Londoño,LFQ,MS
2,564,22-7-22,25-7-22,Madre Laura,1000 Gr,1.0,Natalia Londoño,LFQ,MS
3,566,22-7-22,25-7-22,Madre Laura,2500 Gr,2.0,Mateo Duque,LFQ,MS
4,567,22-7-22,26-7-22,Madre Laura,2500 Gr,1.0,Akio,LFQ,MS
5,568,25-7-22,26-7-22,Madre Laura,2500 Gr,3.0,Juan Camilo Baez,LFQ,MS


Revisemos si en este caso tenemos el mismo comportamiento de columnas que en el archivo de tostión, es decir, si las columnas tienen espacios adicionales o caracteres no deseados. Si es así, debemos limpiarlas para evitar problemas al acceder a ellas. Además, debemos asegurarnos de que las fechas estén en un formato consistente para facilitar su manejo y análisis posterior.

In [50]:
print(data_despachos.columns.tolist())

['# PEDIDO ', 'FECHA DE TUESTE ', 'FECHA DE EMPAQUE ', 'TIPO DE CAFÉ   ', 'PRESENTACIÓN ', 'CANTIDAD ', 'CLIENTE ', 'RESPONSABLE DESPACHO ', 'VERIFICA ']


In [52]:
data_despachos.columns = data_despachos.columns.str.strip()

data_despachos['FECHA DE TUESTE'] = pd.to_datetime(data_despachos['FECHA DE TUESTE'], dayfirst=True, errors='coerce')
data_despachos['FECHA DE EMPAQUE'] = pd.to_datetime(data_despachos['FECHA DE EMPAQUE'], dayfirst=True, errors='coerce')

data_despachos.head()

,# PEDIDO,FECHA DE TUESTE,FECHA DE EMPAQUE,TIPO DE CAFÉ,PRESENTACIÓN,CANTIDAD,CLIENTE,RESPONSABLE DESPACHO,VERIFICA
1,564,2022-07-22,2022-07-25,Madre Laura,500 Gr,3.0,Natalia Londoño,LFQ,MS
2,564,2022-07-22,2022-07-25,Madre Laura,1000 Gr,1.0,Natalia Londoño,LFQ,MS
3,566,2022-07-22,2022-07-25,Madre Laura,2500 Gr,2.0,Mateo Duque,LFQ,MS
4,567,2022-07-22,2022-07-26,Madre Laura,2500 Gr,1.0,Akio,LFQ,MS
5,568,2022-07-25,2022-07-26,Madre Laura,2500 Gr,3.0,Juan Camilo Baez,LFQ,MS


### Análisis de los archivos

Ya obtenidos los DataFrames de los archivos de Excel, y limpiados los nombres de las columnas y los tipos de algunas columnas, procedemos a realizar un análisis exploratorio de los datos, entre estos, revisaremos a que hace referencia cada columna, si tiene información relevante, si es necesario eliminarla o transformarla, y si es necesario realizar algún tipo de preprocesamiento adicional. Adicionalmente, revisaremos cual de esta información es posible usar para nuestro objetivo de predecir el Puntaje de Taza, para así evitar filtrar información al momento de elegir las variables de entrada (X) y la variable objetivo (Y).